# Coding Applications in Medicine: Data Science - Part 2

Module adapted from Kaggle: https://www.kaggle.com/code/mariapushkareva/medical-insurance-cost-with-linear-regression/notebook

Dataset source: https://github.com/stedy/Machine-Learning-with-R-datasets

## Introduction

Data Science is a multidisciplinary field that integrates computation, math/statistics, and domain knowledge to understand the world and solve problems. 

The data science lifecycle consists of:
1. Question/problem formulation
2. Data acquisition and cleaning
3. Exploratory data analysis and visualization
4. Prediction and inference

In this notebook, we will explore a sample data on medical insurance and practice statistical analysis and visualization (with minimal math incorporation).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency

In [ ]:
insuranceDF = pd.read_csv("data/insurance.csv")
insuranceDF["smoker numerical"] = (insuranceDF["smoker"] == "yes").astype(int)
insuranceDF

## Data Visualization

The goals of data visualization are to help with your own understanding of the data/results and to communicate the results/conclusions to others. Data can be visualized in many different ways. Below are some points to keep in mind as you design your visualization of your data.

- Keep axis scales consistent. (Consider the scale when comparing similar data).
- Choose axis limits that fills the visualization. (Multiple plots can be used to show different areas of interests).
- Use a perceptually uniform color map. (Use color to highlight data type).
- Markings plays a significant role in the accuracy of our judgements. (Avoid using pie charts, area charts, word cloud, and stacked charts).
- Use conditioning to aid comparison.
- The chosen visual metric tells the story.

There are many components that make up a figure and many different ways to format the visualization. Below is an example of how one might format their visualization.

In [ ]:
fig = sns.scatterplot(data=insuranceDF, x="bmi", y="charges", hue="smoker")
plt.title("Relationship between BMI and Insurance Charges")
plt.xlabel("BMI")
plt.ylabel("Charges")
plt.xlim(10, 60)
plt.ylim(0, 70000)
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5), title="Smoker")
plt.figtext(0.5, -0.05, "Figure 1", ha="center")
plt.show(fig)

For simplicity, all the plots below will be shown as default without additional format changes.

A bar graph is used to compare qualitative variables. 

In [ ]:
# A countplot is a specific form of bar plot that plots the number of occurrences instead of a pre-defined number.
sns.countplot(data=insuranceDF, x="region");

In [ ]:
# Note: The axis in which the data is plot can be changed. 
sns.countplot(data=insuranceDF, y="region", hue="sex");

A histogram is used to determine the distribution of quantitative variables.

In [ ]:
sns.histplot(data=insuranceDF, x="charges", binwidth=1000);

In [ ]:
# The kde parameter for hisplot will produce a smooth density curve along with the histogram
sns.histplot(data=insuranceDF, x="charges", hue="sex", binwidth=1000, kde=True);

A box plot is another method to compare distribution.

In [ ]:
sns.boxplot(data=insuranceDF, x="region", y="charges");

In [ ]:
sns.boxplot(data=insuranceDF, x="region", y="charges", hue="sex");

A violin plot is essentially a box plot, but allows for better comparison of distribution of each variable.

In [ ]:
sns.violinplot(data=insuranceDF, x="region", y="charges", hue="sex");

A line plot is used to depict relationships between pairs of numerical variables.

In [ ]:
# Calculate the mean charge by age
meanChargeByAgeDF = (insuranceDF.groupby("age").agg("mean")
                                .reset_index()[["age", "charges"]]
                                .rename(columns={"charges": "mean charges"}))

sns.lineplot(data=meanChargeByAgeDF, x="age", y="mean charges");

In [ ]:
# Calculate the mean charge by age and sex
meanChargeByAgeSexDF = (insuranceDF.groupby(["age", "sex"]).agg("mean")
                                   .reset_index()[["age", "sex", "charges"]]
                                   .rename(columns={"charges": "mean charges"}))
    
sns.lineplot(data=meanChargeByAgeSexDF, x="age", y="mean charges", hue="sex");

A scatter plot is another method to depict relationships between pairs of numerical variables. Generally speaking, a scatter plot is a better choice for visualization compared to line plots.

In [ ]:
sns.scatterplot(data=meanChargeByAgeDF, x="age", y="mean charges");

In [ ]:
sns.scatterplot(data=meanChargeByAgeSexDF, x="age", y="mean charges", hue="sex");

For more information, check the following:
- User Guide (Seaborn): https://seaborn.pydata.org/tutorial.html
- User Guide (Matplotlib): https://matplotlib.org/stable/tutorials/index
- API Reference (Seaborn): https://seaborn.pydata.org/api.html
- API Reference (Matplotlib): https://matplotlib.org/stable/api/index.html
- Examples (Seaborn): https://seaborn.pydata.org/examples/index.html
- Examples (Matplotlib): https://matplotlib.org/stable/gallery/index.html

## Linear Regression

Linear regression is one way to model the relationship between a dependent variable and one or multiple independent variable. When constructing a linear model, we are attempting to fit the data into the formula $\hat{Y}=X\theta$, where $\hat{Y}$ is the prediction vector, $X$ is the design matrix, and $\theta$ is the parameter vector. There are different approaches to create such model (or equation) mathematically. 

The goal is to use the existing data $X$ and $Y$ data to generate the optimal $\theta$ used for later prediction.

Note: $\hat{Y}$ is a matrix of size $n\times1$. $X$ is a matrix of size $n\times(p+1)$. $\theta$ is a matrix of size $(p+1)\times n$ 

You can think of $\hat{Y}=X\theta$ as the classic $y = mx + b$ but now expanded to cover multiple $x$ (independent variables) and to make predictions $y$ (dependent variable) for multiple people at once.

Suppose you wish to explore the relationship between age and insurance charges.

In [ ]:
sns.scatterplot(data=insuranceDF, x="age", y="charges");

What would we expect based on the plot?

In [ ]:
# Split our dataset to training and testing datasets
linData1_tr, linData1_te = train_test_split(insuranceDF, test_size=0.1, random_state=42)
linData1_tr.reset_index(inplace=True, drop=True)
linData1_te.reset_index(inplace=True, drop=True)

# Create the X and Y matrix for model training.
linTraining1X = linData1_tr[["age"]].to_numpy()
linTraining1Y = linData1_tr["charges"].to_numpy()

# Creating the linear regression model
linModel1 = LinearRegression(fit_intercept=True)
linModel1.fit(linTraining1X, linTraining1Y)

# Resulting linear regression model parameters
linModel1.intercept_, linModel1.coef_

In [ ]:
# Plot of the data with the linear regression line

sns.scatterplot(data=insuranceDF, x="age", y="charges")
# Plot of the linear regression line
plt.plot(np.linspace(18, 64, 1000), 
         linModel1.predict(np.reshape(np.linspace(18, 64, 1000), (-1, 1))), color="red");

In [ ]:
# Use the model to make predictions (Age)
linModel1.predict([[25]])

In [ ]:
# The score tells you the accuracy of the regression model
# You should use the test dataset for the score calculation
linModel1.score(linData1_te[["age"]].to_numpy(), linData1_te["charges"].to_numpy())

This time, we want to explore the relationship between all the numerical variables and insurance charges.

In [ ]:
# Scatterplot capturing four features/dimensions
sns.scatterplot(data=insuranceDF, x="age", y="charges", hue="smoker", size="bmi");
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5));

What would we expect based on the plot?

In [ ]:
# Split our dataset to training and testing datasets
linData2_tr, linData2_te = train_test_split(insuranceDF, test_size=0.1, random_state=24)
linData2_tr.reset_index(inplace=True, drop=True)
linData2_te.reset_index(inplace=True, drop=True)

# Create the X and Y matrix for model training.
linTraining2X = linData2_tr.drop(["sex", "charges", "smoker", "region"], axis=1).to_numpy()
linTraining2Y = linData2_tr["charges"].to_numpy()

# Creating the linear regression model
linModel2 = LinearRegression(fit_intercept=True)
linModel2.fit(linTraining2X, linTraining2Y)

# Resulting linear regression model parameters
linModel2.intercept_, linModel2.coef_

In [ ]:
# Use the model to make predictions (Age, BMI, # Children, Smoker - 0 or 1)
linModel2.predict([[30, 25, 0, 0]])

In [ ]:
# The score tells you the accuracy of the regression model
# You should use the test dataset for the score calculation
linModel2.score(linData2_te.drop(["sex", "charges", "smoker", "region"], axis=1).to_numpy(),
                linData2_te["charges"].to_numpy())

For more information, check the following:
- User Guide: https://scikit-learn.org/stable/modules/linear_model.html
- API Reference: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model
- Examples: https://scikit-learn.org/stable/auto_examples/index.html#generalized-linear-models

## Logistic Regression

Logistic regression is a model used to estimate the probability that an event occurred given previous data. When the probability exceeds the threshold value, we considered the event as having occurred. Essentially, logistic regression is used for binary (True/False) classification. This is done by the formula $\hat{P}_\theta (Y = 1|x) = \sigma(x^T\theta)$, where $\theta$ is the linear parameter, $\sigma$ is the sigmoid or logistic function, $x$ is the input feature matrix, and $y$ is the response vector.

Suppose you wish to determine if you can predict whether a person is a smoker based on their insurance charges.

In [ ]:
sns.histplot(data=insuranceDF, x="charges", hue="smoker", binwidth=1000, kde=True);

What would we expect based on the plot?

In [ ]:
# A stripplot is a categorical scatterplot that uses jittering to reduce over plotting
sns.stripplot(data=insuranceDF, x="charges", y="smoker numerical", jitter=0.1, orient='h').invert_yaxis()

Here is a different plot depicting the same data.

In [ ]:
# Split our dataset to training and testing datasets
logData1_tr, logData1_te = train_test_split(insuranceDF, test_size=0.1, random_state=21)
logData1_tr.reset_index(inplace=True, drop=True)
logData1_te.reset_index(inplace=True, drop=True)

# Create the X and Y matrix for model training.
logTraining1X = logData1_tr[["charges"]].to_numpy()
logTraining1Y = logData1_tr["smoker numerical"].to_numpy()

# Creating the logistic regression model
logModel1 = LogisticRegression(fit_intercept=True)
logModel1.fit(logTraining1X, logTraining1Y)

# Resulting logistic regression model parameters
logModel1.intercept_, logModel1.coef_

In [ ]:
# Sigmoid or logistic function
def sigmoid(x):
    return 1/(1 + np.exp(-x))

log_theta_0 = logModel1.intercept_[0]
log_theta_1 = logModel1.coef_[0]

# Does the prediction match the actual data
log_smoke_pred_match = logModel1.predict(logData1_tr[["charges"]].to_numpy())==logData1_tr["smoker numerical"]

# Plot of the training data
sns.stripplot(x=logData1_tr["charges"], y=logData1_tr["smoker numerical"],
              jitter=0.1, orient='h', hue=log_smoke_pred_match).invert_yaxis()

# Plot of the logistic regression line
sns.lineplot(x=np.linspace(0, 70000, 1000), y=sigmoid(log_theta_0 + log_theta_1 * np.linspace(0, 70000, 1000)));

In [ ]:
# Use the model to make predictions (Charges)
logModel1.predict([[30000]])

In [ ]:
# The score tells you the accuracy of the regression model
# You should use the test dataset for the score calculation
logModel1.score(logData1_te[["charges"]].to_numpy(), logData1_te["smoker numerical"].to_numpy())

This time, we will try to predict if someone is a smoker using all the numerical variables.

In [ ]:
# Scatterplot with introduction of jitter
sns.scatterplot(data=insuranceDF.assign(smoker_jitter=insuranceDF["smoker numerical"] 
                                        + np.random.uniform(-0.2, 0.2, len(insuranceDF))),
                x="charges", y="smoker_jitter", hue="age", size="bmi", style="children",
                sizes=(20, 200))#.invert_yaxis()
plt.legend(loc="center left", bbox_to_anchor=(1, 0.5));

In [ ]:
# Split our dataset to training and testing datasets
logData2_tr, logData2_te = train_test_split(insuranceDF, test_size=0.1, random_state=12)
logData2_tr.reset_index(inplace=True, drop=True)
logData2_te.reset_index(inplace=True, drop=True)

# Create the X and Y matrix for model training.
logTraining2X = logData2_tr.drop(["sex", "smoker", "region", "smoker numerical"], axis=1).to_numpy()
logTraining2Y = logData2_tr["smoker numerical"].to_numpy()

# Creating the logistic regression model
logModel2 = LogisticRegression(fit_intercept=True)
logModel2.fit(logTraining2X, logTraining2Y)

# Resulting logistic regression model parameters
logModel2.intercept_, logModel2.coef_

In [ ]:
# Use the model to make predictions (Age, BMI, # Children, Insurance Charges)
logModel2.predict([[30, 25, 0, 3000]])

In [ ]:
# The score tells you the accuracy of the regression model
# You should use the test dataset for the score calculation
logModel2.score(logData2_te.drop(["sex", "smoker", "region", "smoker numerical"], axis=1).to_numpy(), 
                logData2_te["smoker numerical"].to_numpy())

For more information, check the following:
- User Guide: https://scikit-learn.org/stable/modules/linear_model.html
- API Reference: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model
- Examples: https://scikit-learn.org/stable/auto_examples/index.html#generalized-linear-models

## Hypothesis Testing

A hypothesis test is used to decide whether the data supports a particular hypothesis. The type of hypothesis testing done typically depends on the type of data that is being used. 

1. Suppose you wish to determine whether there is a significant difference between the proportions of male/female sex in smokers and non-smokers. To answer this question, we will perform a chi-squared test of independence.

In [ ]:
sns.countplot(data=insuranceDF, y="smoker", hue="sex");

What would we expect based on the plot?

In [ ]:
# Contigency table by count 
smokerSexContigency = pd.crosstab(insuranceDF["smoker"], insuranceDF["sex"])

In [ ]:
# Chi-squared analysis
chi2_contingency(smokerSexContigency, correction=False)

What can we conclude based on the calculated statistic?

2. Suppose you wish to determine whether there is a significant difference in insurance charges between the smoker and non-smokers. To answer this question, we will perform a two-sample t-test.

In [ ]:
sns.histplot(data=insuranceDF, x="charges", hue="smoker", binwidth=1000, kde=True);

What would we expect based on the plot?

In [ ]:
sns.boxplot(data=insuranceDF, x="smoker", y="charges");

Here is a different plot depicting the same data.

In [ ]:
# Stratification by smoker/non-smoker
smokerGroup = insuranceDF[insuranceDF["smoker"]=="yes"]
nonsmokerGroup = insuranceDF[insuranceDF["smoker"]=="no"]

# T-test analysis
ttest_ind(smokerGroup["charges"], nonsmokerGroup["charges"])

What can we conclude based on the calculated statistic?

For more information, check the following:
- User Guide: https://docs.scipy.org/doc/scipy/tutorial/stats.html
- API Reference: https://docs.scipy.org/doc/scipy/reference/stats.html

## Practice: Putting it all together

We will analyze the Wisconsin Breast Cancer Dataset from scikit learn. Our goal is to create a model to determine whether a tumor is malignant or benign.

In [ ]:
import sklearn.datasets

breastCancer_dict = sklearn.datasets.load_breast_cancer()
breastCancerDF = pd.DataFrame(breastCancer_dict["data"], columns=breastCancer_dict["feature_names"])
breastCancerDF

In [ ]:
breastCancer_dict["target_names"]

Below is the list of all the features provided by the data. Which feature do you think would be good to use to determine whether a tumor is benign or malignant?

In [ ]:
breastCancerDF.columns

Take a look at the provided encoded labels/values for the malignant column data. It looks like 0 represents malignant and 1 represents benign. We should reverse this encoding of the value to make it more intuitive before adding it as a new column to our data frame.

In [ ]:
# Hint: Currently, breastCancer_dict["target"] store the data where 0 is malignant and 1 is benign. 

### breastCancerDF["malignant"] = ________

Based on the one feature you choose, make an appropriate visualization.

In [ ]:
# Hint: In this case, "malignant" column would be the dependent variable


Looking at your visualization, can you identify a pattern that help you predict whether a tumor is benign or malignant?

Fit the appropriate model to help make the prediction.

In [ ]:
# Step 1: Split our dataset to training and testing datasets

### breastCancerDF_tr, breastCancerDF_te = ________
### breastCancerDF_tr.reset_index(inplace=True, drop=True)
### breastCancerDF_te.reset_index(inplace=True, drop=True)

# Step 2: Create the X and Y matrix for model training.

### breastCancerDF_trX = ________
### breastCancerDF_trY = ________

# Step 3: Creating the model

### breastCancerAreaModel = ________
### breastCancerAreaModel.________(________, ________);

In [ ]:
# Determine the accuracy of the model

### breastCancerAreaModel.________(________, ________)

In [ ]:
# Try using the model to make predictions

### breastCancerAreaModel.________(________)

Now we want to see if there is a significant difference in the feature you selected between benign and malignant tumors.

Based on the one feature you choose, make an appropriate visualization.

In [ ]:
# Hint: In this case, "malignant" column would be the independent variable


Looking at your visualization, can you determine whether there is a significant difference between benign or malignant tumors?

Calculate the test statistic using an appropriate test.

In [ ]:
# Step 1: Create the appropriate grouping or table

### benignGroup = ________
### tumorGroup = ________

# Step 2: Use the appropriate test for analysis

### ________(________, ________)

Below is one way to approach this problem.

In [ ]:
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#


Logistic Regression

In [ ]:
breastCancer_dict = sklearn.datasets.load_breast_cancer()
breastCancerDF = pd.DataFrame(breastCancer_dict["data"], columns=breastCancer_dict["feature_names"])

In [ ]:
# Example using comparison operator to produce T/F values which is then converted to 1/0
### breastCancerDF["malignant"] = (breastCancer_dict["target"] == 0).astype(int)

# Example using mathematical manipulations 
breastCancerDF["malignant"] = 1 - breastCancer_dict["target"]

In [ ]:
sns.histplot(data = breastCancerDF, x = "mean area", hue = "malignant", binwidth=50, kde=True)
plt.legend(labels = ["Malignant", "Benign"]);

In [ ]:
breastCancerDF_tr, breastCancerDF_te = train_test_split(breastCancerDF, test_size=0.10, random_state=33)
breastCancerDF_tr.reset_index(inplace=True, drop=True)
breastCancerDF_te.reset_index(inplace=True, drop=True)

breastCancerDF_trX = breastCancerDF_tr[["mean area"]].to_numpy()
breastCancerDF_trY = breastCancerDF_tr["malignant"].to_numpy()

breastCancerAreaModel = LogisticRegression(fit_intercept=True)
breastCancerAreaModel.fit(breastCancerDF_trX, breastCancerDF_trY);

In [ ]:
breastCancerAreaModel.score(breastCancerDF_te[["mean area"]].to_numpy(), 
                            breastCancerDF_te[["malignant"]].to_numpy())

In [ ]:
breastCancerAreaModel.predict([[1500]])

Two Sample t-Test

In [ ]:
sns.boxplot(data=breastCancerDF, x="malignant", y="mean area");

In [ ]:
benignGroup = breastCancerDF[breastCancerDF["malignant"] == 0]
tumorGroup = breastCancerDF[breastCancerDF["malignant"] == 1]

ttest_ind(benignGroup["mean area"], tumorGroup["mean area"])